In [3]:
import requests
from bs4 import BeautifulSoup

In [4]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_content = requests.get(url).text
soup = BeautifulSoup(html_content,"lxml")

postal_table is the whole table while postal_table_data has the contents of the rows

In [5]:
postal_table = soup.find("table",attrs = {"class":"wikitable"})
postal_table_data = postal_table.tbody.find_all("tr")

postal_table_headers has the contents of the first row which is the label for the columns

In [6]:
postal_table_headers = []
for th in postal_table_data[0].find_all("th"):
    postal_table_headers.append(th.text.replace('\n',' ').strip())

postal_code,borough,neighborhood are the lists that contain the data of the rows

In [7]:
postal_code = []
borough = []
neighborhood = []
for i in range(1,104):
    d = postal_table_data[i].find_all("td")
    postal_code.append(d[0].text.replace('\n', ' ').strip())
    borough.append(d[1].text.replace('\n', ' ').strip())
    neighborhood.append(d[2].text.replace('\n', ' ').strip())

In [8]:
import pandas as pd

In [9]:
df = pd.DataFrame()  
df[postal_table_headers[0]] = postal_code
df[postal_table_headers[1]] = borough
df[postal_table_headers[2]] = neighborhood

In [11]:
df = df[df['Borough'] != 'Not assigned']